In [1]:
import org.apache.spark.rdd.RDD
import org.apache.hadoop.io.Text
import org.apache.hadoop.io.IntWritable
import scala.collection.Seq
import org.apache.hadoop.io._
import scala.collection.mutable.ListBuffer

Intitializing Scala interpreter ...

Spark Web UI available at http://cluster-7e2e-m:8088/proxy/application_1648817619633_0006
SparkContext available as 'sc' (version = 3.1.2, master = yarn, app id = application_1648817619633_0006)
SparkSession available as 'spark'


import org.apache.spark.rdd.RDD
import org.apache.hadoop.io.Text
import org.apache.hadoop.io.IntWritable
import scala.collection.Seq
import org.apache.hadoop.io._
import scala.collection.mutable.ListBuffer


In [2]:
class count_process {
   var new_pairs = 0
   def iterate_1(rdd:RDD[(Int, Int)]):RDD[(Int, Int)] = {
     val rdd_1 = rdd.flatMap(x => List((x._1, x._2), (x._2, x._1))).groupByKey().map(x => (x._1, x._2, x._2.min)).filter(x => x._1 > x._3).map(x=> (x._1, x._2.filter(_ != x._3), x._3))
     new_pairs = rdd_1.map(_._2.size).sum().toInt
                            
     return rdd_1.flatMap(x => List((x._1, x._3)) ++ x._2.map(y => (y, x._3))).distinct()
   }
  
   def iterate_2(rdd:RDD[(Int, Int)]):RDD[(Int, Int)] = {
     val rdd_2 = rdd.flatMap(x => List((x._1, x._2), (x._2, x._1))).groupByKey().mapValues(_.toList.sorted).filter(x => x._1 > x._2(0)).map(x => (x._1, x._2.slice(1, x._2.size), x._2(0)))
    
     new_pairs = rdd_2.map(_._2.size).sum().toInt
                            
     return rdd_2.flatMap(x => List((x._1, x._3)) ++ x._2.map(y => (y, x._3))).distinct()
    
   }
  
   def process(rdd:RDD[(Int, Int)], secondary_sorting: Boolean): RDD[(Int, Int)] = {
     var i = 0
     var rdd_3 = rdd
     do {
       if (secondary_sorting) {
         rdd_3 = this.iterate_2(rdd_3)
       } else {
         rdd_3 = this.iterate_1(rdd_3)
       }
       i+=1
     }
     while(new_pairs > 0)
     return rdd_3
   }
 }


defined class count_process


In [3]:
val algo = new count_process()

algo: count_process = count_process@37e4a749


### facebook_combined

In [4]:
val fb = sc.textFile("gs://pjt_bucket1/data1/facebook_combined.txt.gz").
    map(x => (x.split(" ")(0).toInt, x.split(" ")(1).toInt))
fb.take(5)

fb: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[2] at map at <console>:33
res0: Array[(Int, Int)] = Array((0,1), (0,2), (0,3), (0,4), (0,5))


In [5]:
val t1 = System.nanoTime()
val fb2 = algo.process(fb, false)
println("Execution time : " + ((System.nanoTime()-t1)*10e-10).toString + " s.")

Execution time :4.567925648 s.


t1: Long = 14194118103149
fb2: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[68] at distinct at <console>:37


### twitter_combined

In [6]:
val tw = sc.textFile("gs://pjt_bucket1/data1/twitter_combined.txt.gz").
    map(x => (x.split(" ")(0).toInt, x.split(" ")(1).toInt))
tw.take(5)

tw: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[71] at map at <console>:33
res2: Array[(Int, Int)] = Array((214328887,34428380), (17116707,28465635), (380580781,18996905), (221036078,153460275), (107830991,17868918))


In [7]:
val t1 = System.nanoTime()
val tw2 = algo.process(tw, false)
println("Execution time : " + ((System.nanoTime()-t1)*10e-10).toString + " s.")

Execution time : 22.768157336 s.


t1: Long = 14202391750770
tw2: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[137] at distinct at <console>:37


### google-web

In [8]:
val gg = sc.textFile("gs://pjt_bucket1/data1/web-Google2.txt.gz").
    map(x => x.split('\t')).
    map(x => (x(0).toInt, x(1).toInt))
gg.take(5)

gg: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[141] at map at <console>:34
res4: Array[(Int, Int)] = Array((0,11342), (0,824020), (0,867923), (0,891835), (11342,0))


In [9]:
val t1 = System.nanoTime()
val gg2 = algo.process(gg, false)
println("Execution time : " + ((System.nanoTime()-t1)*10e-10).toString + " s.")

Execution time : 70.886819535 s.


t1: Long = 14227139895657
gg2: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[229] at distinct at <console>:37
